In [1]:
#new code

import time
import tkinter as tk

# Simplified Dungeon Grid
dungeon = [
    ['S', '.', '.', '.','.','#', '.','.','.','.','.'],
    ['#', '#', '.', '.','.','.', '.','.','X','.','#'],
    ['.', '#', '.', '.','.','#', '.','.','.','.','.'],
    ['.', '#', '.', '.','.','#', '.','.','.','#','.'],
    ['.', '#', '.', '.','.','X', '.','.','.','X','.'],
    ['.', '#', 'X', '.','.','X', '.','.','.','X','.'],
    ['.', '#', '#', '#','#','X', '#','#','.','#','X'],
    ['.', '.', '.', '.','.','.', 'X','#','.','.','.'],
    ['.', '.', '.', '#','.','.', '#','.','.','.','.'],
    ['.', '.', '.', '#','.','.', '#','.','.','.','.'],
    ['.', '.', '.', '#','.','.', 'X','#','.','X','.'],
    ['.', '.', '.', '#','.','.', '#','#','.','.','X'],
    ['.', '.', '.', '#','.','.', '#','.','X','.','G']
]

start = (0, 0)
goal = (12, 10)
cell_size = 30

def bfs_explore_all(grid, start, goal):
    rows, cols = len(grid), len(grid[0])
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    queue = [start]
    visited = set([start])
    parent = {start: None}
    explored = []

    while queue:
        x, y = queue.pop(0)
        explored.append((x, y))

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (0 <= nx < rows and 0 <= ny < cols and 
                (nx, ny) not in visited and grid[nx][ny] != '#'):
                queue.append((nx, ny))
                visited.add((nx, ny))
                parent[(nx, ny)] = (x, y)

    # Find the optimal path avoiding traps and obstacles
    queue = [start]
    visited = set([start])
    path_parent = {start: None}
    while queue:
        x, y = queue.pop(0)
        if (x, y) == goal:
            path = []
            while (x, y) != start:
                path.append((x, y))
                x, y = path_parent[(x, y)]
            path.append(start)
            return path[::-1], explored

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (0 <= nx < rows and 0 <= ny < cols and 
                (nx, ny) not in visited and grid[nx][ny] not in ['#', 'X']):
                queue.append((nx, ny))
                visited.add((nx, ny))
                path_parent[(nx, ny)] = (x, y)

    return None, explored

def visualize_search(grid, path, explored):
    window = tk.Tk()
    window.title("Dungeon Explorer Visualization")
    canvas = tk.Canvas(window, width=len(grid[0]) * cell_size, height=len(grid) * cell_size + 50)
    canvas.pack()

    shadow_offset = 5  # Shadow offset for 3D effect

    def draw_cell(x, y, color, shadow=False):
        """Draws a single cell with or without shadow"""
        x1, y1 = x * cell_size, y * cell_size
        x2, y2 = x1 + cell_size, y1 + cell_size
        if shadow:
            canvas.create_rectangle(x1 + shadow_offset, y1 + shadow_offset, x2 + shadow_offset, y2 + shadow_offset,
                                    fill="gray", outline="red", width=2)
        canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="red")

    # Initialize all the cells
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            color = "white"
            if grid[i][j] == '#':
                color = "black"  # Obstacle
            elif grid[i][j] == 'S':
                color = "green"  # Start
            elif grid[i][j] == 'G':
                color = "red"  # Goal
            elif grid[i][j] == 'X':
                color = "orange"  # Trap

            draw_cell(j, i, color)

    # Add a text area for displaying time and cells explored
    info_label = tk.Label(window, text="", font=("Arial", 14), anchor="w", justify="left")
    info_label.pack()

    # Animate explored cells
    explored_idx = 0
    start_time = time.time()

    def animate_explored():
        nonlocal explored_idx
        if explored_idx < len(explored):
            x, y = explored[explored_idx]
            draw_cell(y, x, "lightblue", shadow=True)
            explored_idx += 1
            elapsed_time = time.time() - start_time
            info_label.config(
                text=f"Time Elapsed: {elapsed_time:.2f} seconds\nCells Explored: {explored_idx}")
            window.after(50, animate_explored)  # Redraw after 50ms to create an animation effect

    # Animate path cells
    path_idx = 0

    def animate_path():
        nonlocal path_idx
        if path_idx < len(path):
            x, y = path[path_idx]
            draw_cell(y, x, "yellow", shadow=True)
            path_idx += 1
            elapsed_time = time.time() - start_time
            info_label.config(
                text=f"Path Found in: {elapsed_time:.2f} seconds\nCells Explored: {len(explored)}\nPath Length: {len(path)}")
            window.after(150, animate_path)  # Redraw after 150ms to create an animation effect

    # Start the animation for explored cells first
    animate_explored()

    # Once the explored animation finishes, animate the path
    def start_path_animation():
        window.after(50 * len(explored), animate_path)  # Start path animation after a short delay

    window.after(50 * len(explored), start_path_animation)

    window.mainloop()

# Run BFS and visualize
path, explored = bfs_explore_all(dungeon, start, goal)
if path is None:
    print("No path found between start and goal!")
else:
    print(f"Path found: {path}")
visualize_search(dungeon, path, explored)

Path found: [(0, 0), (0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4)]
